<a href="https://colab.research.google.com/github/pavass/Audiobooks_churn_modelling/blob/master/Audiobooks_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import tensorflow as tf


# We will use the sklearn preprocessing library, as it will be easier to standardize the data.
from sklearn import preprocessing

# Load the data
raw_csv_data = np.loadtxt('original.csv',delimiter=',')

# The inputs are all columns in the csv, except for the first one [:,0]
# (which is just the arbitrary customer IDs that bear no useful information),
# and the last one [:,-1] (which is our targets)

unscaled_inputs_all = raw_csv_data[:,1:-1]

# The targets are in the last column. That's how datasets are conventionally organized.
targets_all = raw_csv_data[:,-1]

##Balance the dataset


In [0]:
# Count how many targets are 1 (meaning that the customer did convert)
num_one_targets = int(np.sum(targets_all))

# Set a counter for targets that are 0 (meaning that the customer did not convert)
zero_targets_counter = 0

# We want to create a "balanced" dataset, so we will have to remove some input/target pairs.
# Declare a variable that will do that:
indices_to_remove = []

# Count the number of targets that are 0. 
# Once there are as many 0s as 1s, mark entries where the target is 0.
for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

# Create two new variables, one that will contain the inputs, and one that will contain the targets.
# We delete all indices that we marked "to remove" in the loop above.
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

##Standardize the inputs

In [0]:
# That's the only place we use sklearn functionality. We will take advantage of its preprocessing capabilities
# It's a simple line of code, which standardizes the inputs, as we explained in one of the lectures.
# At the end of the business case, you can try to run the algorithm WITHOUT this line of code. 
# The result will be interesting.
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

##Shuffle the data

In [0]:
# When the data was collected it was actually arranged by date
# Shuffle the indices of the data, so the data is not arranged in any way when we feed it.
# Since we will be batching, we want the data to be as randomly spread out as possible
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

# Use the shuffled indices to shuffle the inputs and targets.
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

##Split the dataset into train,validation and test

In [0]:
# Count the total number of samples
samples_count = shuffled_inputs.shape[0]

# Count the samples in each subset, assuming we want 80-10-10 distribution of training, validation, and test.
# Naturally, the numbers are integers.
train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)

# The 'test' dataset contains all remaining data.
test_samples_count = samples_count - train_samples_count - validation_samples_count

# Create variables that record the inputs and targets for training
# In our shuffled dataset, they are the first "train_samples_count" observations
train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

# Create variables that record the inputs and targets for validation.
# They are the next "validation_samples_count" observations, folllowing the "train_samples_count" we already assigned
validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

# Create variables that record the inputs and targets for test.
# They are everything that is remaining.
test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]
# We balanced our dataset to be 50-50 (for targets 0 and 1), but the training, validation, and test were 
# taken from a shuffled dataset. Check if they are balanced, too. Note that each time you rerun this code, 
# you will get different values, as each time they are shuffled randomly.
# Normally you preprocess ONCE, so you need not rerun this code once it is done.
# If you rerun this whole sheet, the npzs will be overwritten with your newly preprocessed data.

# Print the number of targets that are 1s, the total number of samples, and the proportion for training, validation, and test.
print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1809.0 3579 0.5054484492875104
214.0 447 0.47874720357941836
214.0 448 0.47767857142857145


##Save the three datasets in .npz

In [0]:
# Save the three datasets in *.npz.
# In the next lesson, you will see that it is extremely valuable to name them in such a coherent way!

np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

In [0]:
npz=np.load('Audiobooks_data_train.npz')
train_inputs=npz['inputs'].astype(np.float)
train_targets=npz['targets'].astype(np.int)
npz=np.load('Audiobooks_data_validation.npz')
validation_inputs,validation_targets=npz['inputs'].astype(np.float),npz['targets'].astype(np.int)
npz=np.load('Audiobooks_data_test.npz')
test_inputs,test_targets=npz['inputs'].astype(np.float),npz['targets'].astype(np.int)

##Model

In [0]:
input_size=10
output_size=2
hidden_layer_size=100
early_stopping=tf.keras.callbacks.EarlyStopping(patience=2)
model=tf.keras.Sequential([
                           tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                           tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                           tf.keras.layers.Dense(output_size,activation='softmax')

                            
])
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
batch_size=100
max_epochs=100
model.fit(train_inputs,train_targets,batch_size=batch_size,epochs=max_epochs,callbacks=[early_stopping],validation_data=(validation_inputs,validation_targets),verbose=2)

Epoch 1/100
36/36 - 0s - loss: 0.5483 - accuracy: 0.7142 - val_loss: 0.4477 - val_accuracy: 0.7472
Epoch 2/100
36/36 - 0s - loss: 0.4150 - accuracy: 0.7742 - val_loss: 0.4005 - val_accuracy: 0.7830
Epoch 3/100
36/36 - 0s - loss: 0.3771 - accuracy: 0.8019 - val_loss: 0.3776 - val_accuracy: 0.7964
Epoch 4/100
36/36 - 0s - loss: 0.3625 - accuracy: 0.8044 - val_loss: 0.3671 - val_accuracy: 0.7987
Epoch 5/100
36/36 - 0s - loss: 0.3531 - accuracy: 0.8069 - val_loss: 0.3679 - val_accuracy: 0.7987
Epoch 6/100
36/36 - 0s - loss: 0.3486 - accuracy: 0.8094 - val_loss: 0.3613 - val_accuracy: 0.7964
Epoch 7/100
36/36 - 0s - loss: 0.3439 - accuracy: 0.8134 - val_loss: 0.3557 - val_accuracy: 0.7919
Epoch 8/100
36/36 - 0s - loss: 0.3453 - accuracy: 0.8106 - val_loss: 0.3674 - val_accuracy: 0.7919
Epoch 9/100
36/36 - 0s - loss: 0.3369 - accuracy: 0.8192 - val_loss: 0.3521 - val_accuracy: 0.8009
Epoch 10/100
36/36 - 0s - loss: 0.3331 - accuracy: 0.8192 - val_loss: 0.3488 - val_accuracy: 0.8076
Epoch 11/

##Test

In [0]:
test_loss,test_accuracy=model.evaluate(test_inputs,test_targets)

14/14 [==============================] - 0s 1ms/step - loss: 0.3342 - accuracy: 0.8237
